In [1]:
from TM_base_calc import *

In [2]:
consideration_list = []
db_list = []
for i in range(1, len(base_cards)):
    if base_cards.Title.iloc[i] in consideration_list:
        db_list.append(base_cards.iloc[i])
db = pd.DataFrame(db_list)
display(db)

""


In [3]:
# scan a variable string to replace it with the variable's value
def vars_scan(benefit_list, generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    benefit_value = 0
    for idx, ele in enumerate(benefit_list):
        if ele in vars_dict_static:
            try:
                benefit_value = int(benefit_list[idx-1]) * vars_dict_static[ele]
            except ValueError: continue
        elif ele in vars_dict_variable:
            benefit_value = int(benefit_list[idx-1]) * vars_dict_variable[ele](income)
        elif ele in vars_dict_resource:
            benefit_value = int(benefit_list[idx-1]) * vars_dict_resource[ele]\
            (generation, player_count)
        elif ele == 'TR' or ele == 'oxygen':
            benefit_value = int(benefit_list[idx-1]) * TR
        elif ele == 'discard':
            benefit_value -= int(benefit_list[idx+1]) * draw
    return benefit_value

In [4]:
# pers

def pers(i_benefit_list, generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
    if 'per' in i_benefit_list:
        per_a, per_b = (' ').join(i_benefit_list).split(' per ')
        per_a, per_b = per_a.split(' '), per_b.split(' ')
        benefit_a = vars_scan(per_a, generation, player_count)
        if 'all' in i_benefit_list: players = player_count
        else: players = 1
            
        if 'tag' in i_benefit_list:
            for idx, ele in enumerate(i_benefit_list):
                if ele in tags_showing_dict:
                    benefit_b = (1 / int(i_benefit_list[idx-2])) * \
                                (tags_showing_dict[ele] * cards_current(generation) * players)
                                
        elif 'city' in i_benefit_list:
            for idx, ele in enumerate(i_benefit_list):
                if ele == 'city':
                    benefit_b = int(i_benefit_list[idx-1]) * \
                                cities_current(generation, player_count)
                    
        elif 'event' in i_benefit_list:
            for idx, ele in enumerate(i_benefit_list):
                if ele == 'event':
                    benefit_b = int(i_benefit_list[idx-1]) * \
                                (chance_event_played * (cards_current(generation) * players))

        points_per = benefit_a * benefit_b
        return points_per

In [5]:
# ors

def ors(or_benefit, generation, player_count, cost=False):
    income, vp, TR, last_gen = initialize(generation, player_count)
    or_list = or_benefit.split(' or ')
    or_value, or_val = [0, 0]
    cost_list = []
    for benefit in or_list:
        benefit = benefit.split(' ')
        or_val = vars_scan(benefit, generation, player_count)
        if cost == True: cost_list.append(or_val)
        else:
            if or_val > or_value: or_value = or_val
    if cost == True: or_value = min(cost_list)
    return or_value

In [6]:
# five recorded games- 2p

Game_1 = pd.read_csv('Game_1.csv')
Game_2 = pd.read_csv('Game_2.csv')
Game_3 = pd.read_csv('Game_3.csv')
Game_4 = pd.read_csv('Game_4.csv')
Game_5 = pd.read_csv('Game_5.csv')

in_depth_game_list = (Game_1, Game_2, Game_3, Game_4, Game_5)
#display(Game_3)

In [7]:
# determine average number of cards played per generation
cards_played_lists = []
for game in in_depth_game_list:
    game_len = (max([int(row) for row in game.Generation if row not in ['Last', 'Final']]))
    p1_played_per_turn = []
    p2_played_per_turn = []
    for gen in range(1, game_len+1):
        p1_cards_played = 0
        p2_cards_played = 0
        for row in game[game.Generation == str(gen)].itertuples():
            if row.Player == 1 and 'played' in row.Action:
                p1_cards_played += 1
            if row.Player == 2 and 'played' in row.Action:
                p2_cards_played += 1
        p1_played_per_turn.append(p1_cards_played)
        p2_played_per_turn.append(p2_cards_played)
    cards_played_lists.append(p1_played_per_turn)
    cards_played_lists.append(p2_played_per_turn)

gen_list = [len(list) for list in cards_played_lists]
max_gen = max(gen_list)

ave_cards_played = []
for gen in range(max_gen):
    played = []
    for list in cards_played_lists:
        try:
            played.append(list[gen])
        except IndexError:
            continue
    ave_cards_played.append(sum(played) / len(played))
    
print('The average number of cards played per generation: ')
for idx, ele in enumerate(ave_cards_played):
    print(f'{idx+1}: {str(ele)}')

The average number of cards played per generation: 
1: 2.0
2: 1.1
3: 2.0
4: 1.4
5: 2.1
6: 2.3
7: 2.5
8: 2.7
9: 3.1
10: 3.0
11: 2.9
12: 4.8
13: 4.6
14: 6.6
15: 6.375
16: 6.666666666666667
17: 6.25


In [8]:
special_case_db = base_cards[base_cards.Title.isin(special_cases)].copy().reset_index(drop=True)
display(special_case_db)

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent
0,Adaptation Technology,base,active,12,NaN,science,NaN,1,NaN,global requirements +/- 2,NaN,NaN,NaN
1,Advanced Alloys,corporate,active,9,NaN,science,NaN,NaN,NaN,steel and titanium are each worth 1 credit extra,NaN,NaN,NaN
2,Anti-Gravity Technology,corporate,active,14,NaN,science,7 science tag,3,NaN,cards cost 2 credit fewer,NaN,NaN,NaN
3,Ants,base,active,9,NaN,microbe,4% oxygen,1 per 2 microbe on card,NaN,NaN,remove 1 microbe from any card,1 microbe on card,1 microbe per turn (active)
4,Arctic Algae,base,active,12,NaN,plant,max -12°C,NaN,1 plant,"when an ocean is placed, 2 plant",NaN,NaN,NaN
5,Business Network,corporate,active,4,1 CREDIT,earth,NaN,NaN,NaN,NaN,NaN,"1 draw, then buy or discard",NaN
6,Capital,base,automated,26,2 ENERGY,"city, building",4 ocean,1 per 1 adjacent ocean,"5 CREDIT, 1 city",NaN,NaN,NaN,NaN
7,CEO’s Favorite Project,corporate,event,1,NaN,event,NaN,NaN,Add 1 resource to a card with at least 1 resource on it,NaN,NaN,NaN,NaN
8,Commercial District,corporate,automated,16,1 ENERGY,building,NaN,1 per 1 adjacent city,"4 CREDIT, special tile",NaN,NaN,NaN,NaN
9,Decomposers,base,active,5,NaN,microbe,3% oxygen,1 per 3 microbe on card,NaN,"when playing an animal, plant, or microbe tag, 1 microbe to card",NaN,NaN,NaN


In [9]:
# category definitions
pricer_list = ['Anti-Gravity Technology', 'Earth Catapult', 'Earth Office', 'Mass Converter', 
           'Quantum Extractor', 'Research Outpost', 'Shuttles', 'Space Station']

#### Projects that reduce the price of other projects
* Anti-Gravity Technology
* Earth Catapult
* Earth Office
* Mass Converter
* Quantum Extractor
* Research Outpost
* Shuttles
* Space Station

##### Notes - 
* An average number of cards played per generation has been determined to begin with 

In [10]:
pricer_db = base_cards[base_cards.Title.isin(pricer_list)].copy().reset_index(drop=True)
display(pricer_db)

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent
0,Anti-Gravity Technology,corporate,active,14,NaN,science,7 science tag,3,NaN,cards cost 2 credit fewer,NaN,NaN,NaN
1,Earth Catapult,corporate,active,23,NaN,earth,NaN,2,NaN,cards cost 2 credit fewer,NaN,NaN,NaN
2,Earth Office,corporate,active,1,NaN,earth,NaN,NaN,NaN,cards with an earth tag cost 3 credit fewer,NaN,NaN,NaN
3,Mass Converter,corporate,active,8,NaN,"science, power",5 science tag,NaN,6 ENERGY,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN
4,Quantum Extractor,corporate,active,13,NaN,"science, power",4 science tag,NaN,4 ENERGY,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN
5,Research Outpost,base,active,18,NaN,"science, city, building",NaN,NaN,1 city next to no other tile,cards cost 1 credit fewer,NaN,NaN,NaN
6,Shuttles,base,active,10,1 ENERGY,space,5% oxygen,1,2 CREDIT,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN
7,Space Station,corporate,active,10,NaN,space,NaN,1,NaN,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN


In [11]:
# price-reducers
pricer_db = base_cards[base_cards.Title.isin(pricer_list)].copy().reset_index(drop=True)

def price_reducers(generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
# iterate through cards in database
    for row in pricer_db.itertuples():
        dict_row = row._asdict()
        value = 0
        
    # Primary Cost------
        value -= dict_row['Primary_Cost']
        
    # Additional Cost------
        add_cost = dict_row['Additional_Cost']
        if not isinstance(add_cost, float):
            add_cost_lists = []
            for idx, ele in enumerate(add_cost.split(',')):
                add_cost_lists.append(ele.strip().split(' '))
            for add_cost_list in add_cost_lists:
                value -= vars_scan(add_cost_list, generation, player_count)
                            
    # Victory Points------
        victory = dict_row['Victory_Points']
        if not isinstance(victory, float):
            if len(victory) <= 2:
                value += int(victory) * vp
                
    # Immediate Benefit------
        i_benefit = dict_row['Immediate_Benefit']
        if not isinstance(i_benefit, float):
            i_benefit_lists = []
            for idx, ele in enumerate(i_benefit.split(',')):
                i_benefit_lists.append(ele.strip().split(' '))
            for idx, i_benefit_list in enumerate(i_benefit_lists):
                
            # for benefits dependent on the amount of another variable
                if 'per' in i_benefit_list:
                    value += pers(i_benefit_list, generation, player_count)
                
            # for benefits with a choice; the larger of the two values is taken
                elif 'or' in i_benefit_list:
                    value += ors((' ').join(i_benefit_list), generation, player_count)
                    
            # for ocean and greenery tiles played on the opposite site
                elif 'land' in i_benefit_list and 'ocean' in i_benefit_list:
                    value += ocean + tile_land - tile_ocean
                elif 'ocean' in i_benefit_list and 'greenery' in i_benefit_list:
                    value += greenery + tile_ocean - tile_land
                    
            # for special tiles
                elif 'special' in i_benefit_list:
                    if 'ocean' in i_benefit_list:
                        value += tile_ocean
                    else:
                        value += tile_land
                
            # for simple added values
                else:
                    value += vars_scan(i_benefit_list, generation, player_count)
                    
    # Passive Benefit------
        p_benefit = dict_row['Passive_Benefit']
        p_benefit_list = p_benefit.split(' ')
        cards_to_play = sum(ave_cards_played[generation-1:last_gen])
        
        if not 'tag' in p_benefit_list:
            value += (cards_to_play - 1) * int(p_benefit_list[2])
            
        elif 'tag' in p_benefit_list:
            for idx, ele in enumerate(p_benefit_list):
                if ele in tags_played_dict:
                    value += 4 * (last_gen - generation + 1) * tags_played_dict[ele] \
                    * int(p_benefit_list[idx+3])
                    
        pricer_db.loc[dict_row['Index'], 'Value'] = round(value)
        pricer_db.loc[dict_row['Index'], 'Current_Generation'] = generation
        pricer_db.loc[dict_row['Index'], 'Players'] = player_count
    pricer_db.Value = pricer_db.Value.astype(int)
    pricer_db.Current_Generation = pricer_db.Current_Generation.astype(int)
    pricer_db.Players = pricer_db.Players.astype(int)
    
    return pricer_db

display(price_reducers(5, 2))

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent,Value,Current_Generation,Players
0,Anti-Gravity Technology,corporate,active,14,NaN,science,7 science tag,3,NaN,cards cost 2 credit fewer,NaN,NaN,NaN,65,5,2
1,Earth Catapult,corporate,active,23,NaN,earth,NaN,2,NaN,cards cost 2 credit fewer,NaN,NaN,NaN,48,5,2
2,Earth Office,corporate,active,1,NaN,earth,NaN,NaN,NaN,cards with an earth tag cost 3 credit fewer,NaN,NaN,NaN,10,5,2
3,Mass Converter,corporate,active,8,NaN,"science, power",5 science tag,NaN,6 ENERGY,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,151,5,2
4,Quantum Extractor,corporate,active,13,NaN,"science, power",4 science tag,NaN,4 ENERGY,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,98,5,2
5,Research Outpost,base,active,18,NaN,"science, city, building",NaN,NaN,1 city next to no other tile,cards cost 1 credit fewer,NaN,NaN,NaN,25,5,2
6,Shuttles,base,active,10,1 ENERGY,space,5% oxygen,1,2 CREDIT,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,7,5,2
7,Space Station,corporate,active,10,NaN,space,NaN,1,NaN,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,13,5,2


In [12]:
### DEBUGGER
# price-reducers
pricer_db = base_cards[base_cards.Title.isin(pricer_list)].copy().reset_index(drop=True)

def price_reducers(generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
  # iterate through cards in database
    for row in pricer_db.itertuples():
        dict_row = row._asdict()
        value = 0
        print('Card - ', dict_row['Title'])
        
    # Primary Cost------
        value -= dict_row['Primary_Cost']
        print('Primary Cost = ', dict_row['Primary_Cost'])
        
    # Additional Cost------
        add_cost = dict_row['Additional_Cost']
        if not isinstance(add_cost, float):
            add_cost_lists = []
            for idx, ele in enumerate(add_cost.split(',')):
                add_cost_lists.append(ele.strip().split(' '))
            for add_cost_list in add_cost_lists:
                value -= vars_scan(add_cost_list, generation, player_count)
            print('Additional Cost = ', vars_scan(add_cost_list, generation, player_count))
        else: print('Additional Cost = None')
                            
    # Victory Points------
        victory = dict_row['Victory_Points']
        if not isinstance(victory, float):
            if len(victory) <= 2:
                value += int(victory) * vp 
                print('Victory Points = ', int(victory) * vp)
            else: print('VP per calculated in Active Benefits')
        else: print('Victory Points = None')
                
    # Immediate Benefit------
        i_benefit = dict_row['Immediate_Benefit']
        if not isinstance(i_benefit, float):
            i_benefit_lists = []
            for idx, ele in enumerate(i_benefit.split(',')):
                i_benefit_lists.append(ele.strip().split(' '))
            for idx, i_benefit_list in enumerate(i_benefit_lists):
                
            # for benefits dependent on the amount of another variable
                if 'per' in i_benefit_list:
                    value += pers(i_benefit_list, generation, player_count)
                    print('Immediate Per Points = ', pers(i_benefit_list, generation, player_count))
                
            # for benefits with a choice; the larger of the two values is taken
                elif 'or' in i_benefit_list:
                    value += ors((' ').join(i_benefit_list), generation, player_count)
                    print('Immediate Or Points = ', ors((' ').join(i_benefit_list), generation, player_count))
                    
            # for ocean and greenery tiles played on the opposite site
                elif 'land' in i_benefit_list and 'ocean' in i_benefit_list:
                    value += ocean + tile_land - tile_ocean
                    print('Opposite Land Type = ', ocean + tile_land - tile_ocean)
                elif 'ocean' in i_benefit_list and 'greenery' in i_benefit_list:
                    value += greenery + tile_ocean - tile_land
                    print('Opposite Land Type = ', greenery + tile_ocean - tile_land)
                    
            # for special tiles
                elif 'special' in i_benefit_list:
                    if 'ocean' in i_benefit_list:
                        value += tile_ocean
                        print('Special Tile Ocean = ', tile_ocean)
                    else:
                        value += tile_land
                        print('Special Tile Land = ', tile_land)
                
            # for simple added values
                else:
                    value += vars_scan(i_benefit_list, generation, player_count)
                    print('Immediate Benefit = ', vars_scan(i_benefit_list, generation, player_count))
        else: print ('Immediate Benefit = None')
                    
    # Active Cost/Benefit------
        act_cost = dict_row['Active_Cost']
        a_benefit = dict_row['Active_Benefit']
        current_gen = generation
        if isinstance(act_cost, float) and isinstance(a_benefit, float):
            print('Active Cost and Benefit = None')
        else:
            total_val = 0
            while current_gen <= last_gen:
                print('***Current Gen = ', current_gen, ' ***')
        
            # Cost
                cost, benefit = [0, 0]
                if not isinstance(act_cost, float):
                    act_cost_list = act_cost.split(' ')
            
                    if 'or' in act_cost_list:
                        cost = ors(act_cost, current_gen, player_count, cost=True)
                        print('Active Or Cost = ', ors(act_cost, current_gen, player_count, cost=True))
                
                    else:
                        if any([x for x in a_benefit.split(' ') if x in vars_dict_variable]):
                            if current_gen < last_gen:
                                cost = vars_scan(act_cost_list, current_gen, player_count)
                                print('Active Cost = ', vars_scan(act_cost_list, current_gen, player_count))
                        else:
                            cost = vars_scan(act_cost_list, current_gen, player_count)
                            print('Active Cost = ', vars_scan(act_cost_list, current_gen, player_count))
                else: print('Active Cost = None')
        
            # Benefit
                if not isinstance(a_benefit, float):
                    benefit = 0
                    for benefit_list in a_benefit.split(','):
                        benefit_list_split = benefit_list.strip().split(' ')
            
                        if 'or' in benefit_list:
                            benefit += ors(benefit_list, current_gen, player_count)
                            print('Active Or Benefit = ', ors(benefit_list, current_gen, player_count))
                
                        elif 'on card' in benefit_list:
                            number = (int(victory[0]) / int(victory[6])) * int(benefit_list[0])
                            current_vp = credits_per_vp(current_gen, player_count)
                            print('current_vp = ', current_vp)
                            benefit += number * current_vp
                            print('Active On Card Benefit = ', number * current_vp)
                
                        else:
                            if any([x for x in benefit_list_split if x in \
                                   [ele for ele in vars_dict_variable if ele \
                                   not in ['CREDIT', 'PLANT']]]):
                            # ensures increased production on the final turn isn't converted into points
                                if current_gen < last_gen:
                                    benefit += vars_scan(benefit_list_split, current_gen, player_count)
                                    print('Active Benefit Variable = ', vars_scan(benefit_list_split, current_gen, \
                                                                        player_count))
                            else:
                                benefit += vars_scan(benefit_list_split, current_gen, player_count)
                                print('Active Benefit Static = ', vars_scan(benefit_list_split, current_gen, player_count))
                else: print('Active Benefit = None')
                if cost < benefit: value += benefit - cost
                print('Net Active Points = ', benefit - cost)
                total_val += benefit - cost
                current_gen += 1
                print('Total Active Cost/Benefit = ', total_val)
                
    # Removal from opponent------
        removed = dict_row['Removed_from_Opponent']
        if not isinstance(removed, float):
            
            if 'or' in removed:
                value += ors(removed, generation, player_count)
                print('Or Removal from Opponent = ', ors(removed, generation, player_count))
            
            else:
                value += vars_scan(removed.split(' '), generation, player_count)
                print('Removal = ', vars_scan(removed.split(' '), generation, player_count))
        else: print('Removal from Opponenet = None')
            
    # Passive Benefit------
        p_benefit = dict_row['Passive_Benefit']
        p_benefit_list = p_benefit.split(' ')
        cards_to_play = sum(ave_cards_played[generation-1:last_gen])
        print('Cards to play: ', cards_to_play)
        
        if not 'tag' in p_benefit_list:
            value += (cards_to_play - 1) * int(p_benefit_list[2])
            print('Passive Benefit = ', (cards_to_play - 1) * int(p_benefit_list[2]))
            
        elif 'tag' in p_benefit_list:
            for idx, ele in enumerate(p_benefit_list):
                if ele in tags_played_dict:
                    value += 4 * (last_gen - generation + 1) * tags_played_dict[ele] \
                    * int(p_benefit_list[idx+3])
                    
        pricer_db.loc[dict_row['Index'], 'Value'] = round(value)
        pricer_db.loc[dict_row['Index'], 'Current_Generation'] = generation
        pricer_db.loc[dict_row['Index'], 'Players'] = player_count
        print('Value = ', value)
    pricer_db.Value = pricer_db.Value.astype(int)
    pricer_db.Current_Generation = pricer_db.Current_Generation.astype(int)
    pricer_db.Players = pricer_db.Players.astype(int)
    return pricer_db

display(price_reducers(5, 2))

Card -  Anti-Gravity Technology
Primary Cost =  14
Additional Cost = None
Victory Points =  24.750335264360093
Immediate Benefit = None
Active Cost and Benefit = None
Removal from Opponenet = None
Cards to play:  28.0
Passive Benefit =  54.0
Value =  64.75033526436009
Card -  Earth Catapult
Primary Cost =  23
Additional Cost = None
Victory Points =  16.500223509573395
Immediate Benefit = None
Active Cost and Benefit = None
Removal from Opponenet = None
Cards to play:  28.0
Passive Benefit =  54.0
Value =  47.500223509573395
Card -  Earth Office
Primary Cost =  1
Additional Cost = None
Victory Points = None
Immediate Benefit = None
Active Cost and Benefit = None
Removal from Opponenet = None
Cards to play:  28.0
Value =  10.423076923076923
Card -  Mass Converter
Primary Cost =  8
Additional Cost = None
Victory Points = None
Immediate Benefit =  144
Active Cost and Benefit = None
Removal from Opponenet = None
Cards to play:  28.0
Value =  150.53846153846155
Card -  Quantum Extractor
Prim

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent,Value,Current_Generation,Players
0,Anti-Gravity Technology,corporate,active,14,NaN,science,7 science tag,3,NaN,cards cost 2 credit fewer,NaN,NaN,NaN,65,5,2
1,Earth Catapult,corporate,active,23,NaN,earth,NaN,2,NaN,cards cost 2 credit fewer,NaN,NaN,NaN,48,5,2
2,Earth Office,corporate,active,1,NaN,earth,NaN,NaN,NaN,cards with an earth tag cost 3 credit fewer,NaN,NaN,NaN,10,5,2
3,Mass Converter,corporate,active,8,NaN,"science, power",5 science tag,NaN,6 ENERGY,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,151,5,2
4,Quantum Extractor,corporate,active,13,NaN,"science, power",4 science tag,NaN,4 ENERGY,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,98,5,2
5,Research Outpost,base,active,18,NaN,"science, city, building",NaN,NaN,1 city next to no other tile,cards cost 1 credit fewer,NaN,NaN,NaN,25,5,2
6,Shuttles,base,active,10,1 ENERGY,space,5% oxygen,1,2 CREDIT,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,7,5,2
7,Space Station,corporate,active,10,NaN,space,NaN,1,NaN,cards with a space tag cost 2 credit fewer,NaN,NaN,NaN,13,5,2


In [13]:
# price-reducers
pricer_db = base_cards[base_cards.Title.isin(pricer_list)].copy().reset_index(drop=True)

@base_value
def price_reducers(database, generation, player_count, row, last_gen):
    value = 0
    p_benefit = row.at['Passive_Benefit']
    p_benefit_list = p_benefit.split(' ')
    cards_to_play = sum(ave_cards_played[(generation-1):last_gen])
    if not 'tag' in p_benefit_list:
        value = (cards_to_play - 1) * int(p_benefit_list[2])
    elif 'tag' in p_benefit_list:
        for idx, ele in enumerate(p_benefit_list):
            if ele in tags_played_dict:
                value = 4 * (last_gen - generation + 1) * tags_played_dict[ele] * int(p_benefit_list[idx+3])
    return value

price_reducers(pricer_db, 11, 2)

NameError: name 'vars_scan' is not defined

In [ ]:
# define value for cards where some arbitrary numbers must be assigned

# Adaptation Technology: 

In [ ]:
# determine current quarter of the game
def game_quarter(generation, player_count):
    quarters_list = []
    for gen in gen_quarters:
        if gen[0] == str(player_count):
            quarters_list.append(gen_quarters[gen])
    absolute_difference_function = lambda list_value : abs(list_value - generation)
    for i in range(4):
        if quarters_list[i] == min(quarters_list, key=absolute_difference_function):
            quarter = i+1
    return quarter

#determine credits per vp based on quarter
def current_cpp(quarter, player_count):    
    for cpp in gen_credits_per_vp:
        if cpp[0] == str(player_count) and cpp[3] == str(quarter):
            vp = gen_credits_per_vp[cpp]
    return vp

#run above functions and determine credits per TR
#def initialize(generation, player_count):
#    income = remgen(generation, player_count)
#    quarter = game_quarter(generation, player_count)
#    vp = current_cpp(quarter, player_count)
#    TR = vp + income
#    return income, quarter, vp, TR

In [ ]:
# Discarded snippets:

# from main page---
# main value calculation function after line 22 (replaces line 23):
                #if i % 2 != 0:    # pulls variable from vars_dict
                #    for var in vars_dict_static:
                #        if var == add_cost_list[i]:
                #            value -= int(add_cost_list[i-1]) * vars_dict_static[var]
                #    for var in vars_dict_variable:
                #        if var == add_cost_list[i]:
                #            value -= int(add_cost_list[i-1]) * vars_dict_variable[var](income)   
                
# main value calculation function after line 47 (replaces line 48):
                    #for var in vars_dict_static:
                    #    if var == i_benefit_list_inner[i][1]:
                    #        value += int(i_benefit_list_inner[i][0]) * vars_dict_static[var]
                    #for var in vars_dict_variable:
                    #    if var == i_benefit_list_inner[i][1]:
                    #        value += int(i_benefit_list_inner[i][0]) * vars_dict_variable[var](income)
                    #for var in vars_dict_resource:
                    #    if var == i_benefit_list_inner[i][1]:
                    #        value += int(i_benefit_list_inner[i][0]) * vars_dict_resource[var]\
                    #        (generation, player_count)
                    #if i_benefit_list_inner[i][1] == 'TR':
                    #    value += int(i_benefit_list_inner[i][0]) * TR
                    
# from this page---
#ors after line 6 (replaces line 7):
        #for var in vars_dict_static:
        #    if var == benefit[1]:
        #        or_val = int(benefit[0]) * vars_dict_static[var]
        #for var in vars_dict_variable:
        #    if var == benefit[1]:
        #        or_val = int(benefit[0]) * vars_dict_variable[var](income)
        #for var in vars_dict_resource:
        #    if var == benefit[1]:
        #        or_val = int(benefit[0]) * vars_dict_resource[var](generation, player_count)
        
#pers after line 6 (replaces line 7):
            #for var in vars_dict_static:
            #    if var == i_benefit_list[i-1]:
            #        benefit_a = int(i_benefit_list[i-2]) * vars_dict_static[var]
            #for var in vars_dict_variable:
            #    if var == i_benefit_list[i-1]:
            #        benefit_a = int(i_benefit_list[i-2]) * vars_dict_variable[var](income)
            #        
            #if i_benefit_list[i-1] == 'TR':
            #    benefit_a = int(i_benefit_list[i-2]) * TR
            
#old active cost/benefit from main function:
# Cost
#        if type(act_cost) != float:
#            act_cost_list = act_cost.split(' ')
#            
#            if 'or' in act_cost_list:
#                value -= ors(act_cost, generation, player_count, cost=True) * \
#                            (income + 1)
#                print('active or cost = ', ors(act_cost, generation, player_count, cost=True) * \
#                      (income + 1))
#                
#            else:
#                if any([x for x in a_benefit.split(' ') if x in vars_dict_variable]):
#                    value -= vars_scan(act_cost_list, generation, player_count) * \
#                         income
#                    print('active cost = ', vars_scan(act_cost_list, generation, player_count) * \
#                         income)
#                else:
#                    value -= vars_scan(act_cost_list, generation, player_count) * \
#                         (income + 1)
#                    print('active cost = ', vars_scan(act_cost_list, generation, player_count) * \
#                          (income + 1))
#        
#        # Benefit
#        if type(a_benefit) != float:
#            a_benefit_list = a_benefit.split(' ')
#            
#            if 'or' in a_benefit:
#                value += ors(a_benefit, generation, player_count) * \
#                         (income + 1)
#                print('active or benefit = ', ors(a_benefit, generation, player_count) * (income + 1))
#                
#            elif 'on card' in a_benefit:
#                vp_list = []
#                for i in range(generation, generation + income + 1):
#                    vp_list.append(credits_per_vp(i, player_count))
#                number = (int(victory[0]) / int(victory[6])) * int(a_benefit[0])
#                value += number * sum(vp_list)
#                print('active on card benefit = ', number * sum(vp_list))
#                
#            else:
#                if any([x for x in a_benefit_list if x in vars_dict_variable]):
#                    gen = generation
#                    total_val = 0
#                    # ensures increased production on the final turn isn't converted into points
#                    while gen < last_gen:
#                        value += vars_scan(a_benefit_list, gen, player_count)
#                        total_val += vars_scan(a_benefit_list, gen, player_count)
#                        print('active benefit variable = ', vars_scan(a_benefit_list, gen, player_count))
#                        gen += 1
#                    print('total abv = ', total_val)
#                else:
#                    value += vars_scan(a_benefit_list, generation, player_count) * \
#                             (income + 1)
#                    print('active benefit static = ', vars_scan(a_benefit_list, generation, player_count) * \
#                             (income + 1))

#### Number of cities played

The average number of cities played in a game is set at 11. This is derived as a result of personal experience through dozens of games (mostly 2-player) as well as a rough averaging from the discussion [here](https://boardgamegeek.com/thread/1861021/how-many-cities).

There are two off-world city areas, and a couple of cards that ignore those cities. Because only 2 cards out of 208 allow a player to play on those areas, the same average value is used for cards that say "Cities" and those that say "Cities ON MARS".

The average number is cities is divided by the average number of generations to determine the number of cities on the board for any given generation and player count.

In [ ]:
#cities_per_game = 11
#Mars_cities_per_game = 10
#
#def cities_current(generation, player_count):
#    for players in last_gens:
#        if str(player_count) in players:
#            last_gen = last_gens[players]
#    cities = (11 / last_gen) * generation
#    return cities
#
#def Mars_cities_current(generation, player_count):
#    for players in last_gens:
#        if str(player_count) in players:
#            last_gen = last_gens[players]
#    Mars_cities = (10 / last_gen) * generation
#    return Mars_cities
#
#print(f'There are an average of {str(round(cities_current(6, 2)))} cities \
#in generation 6 of a 2-player game.')
#print(f'There are an average of {str(round(Mars_cities_current(8, 2)))} cities \
#on Mars in generation 8 of a 2-player game.')

In [ ]:
# determine average number of each terraforming requirement fulfilled per turn
#ocean_gen_lists, oxygen_gen_lists, temperature_gen_lists = [[], [], []]
#for game in in_depth_game_list:
#    ocean_gen_list, oxygen_gen_list, temperature_gen_list = [[], [], []]
#    ocean, oxygen, temperature = [0, 0, 0]
#    game_len = (max([int(row) for row in game.Generation if row not in ['Last', 'Final']]))
#    for gen in range(1, game_len+1):
#        for row in game[game.Generation == str(gen)].itertuples():
#            if not isinstance(row.Terraforming, float):
#                for idx, ele in enumerate(row.Terraforming.split(',')):
#                    ele = ele.strip()
#                    if 'ocean' in ele:
#                        ocean += int(ele[0])
#                    elif 'oxygen' in ele:
#                        oxygen += int(ele[0])
#                    elif 'temperature' in ele:
#                        temperature += int(ele[0])
#        ocean_gen_list.append(ocean)
#        oxygen_gen_list.append(oxygen)
#        temperature_gen_list.append(temperature)
#    ocean_gen_lists.append(ocean_gen_list)
#    oxygen_gen_lists.append(oxygen_gen_list)
#   temperature_gen_lists.append(temperature_gen_list)
#        
#ave_ocean_gen_list, ave_oxygen_gen_list, ave_temperature_gen_list = [[], [], []]
#ave_terraforming_lists = [ave_ocean_gen_list, ave_oxygen_gen_list, ave_temperature_gen_list]
#for gen in range(max_gen):
#    ocean_num_list, oxygen_num_list, temperature_num_list = [[], [], []]
#    new_terraforming_lists = [ocean_num_list, oxygen_num_list, temperature_num_list]
#    all_terraforming_lists = [ocean_gen_lists, oxygen_gen_lists, temperature_gen_lists]
#    for idx, parent_list in enumerate(all_terraforming_lists):
#        for child_list in parent_list:
#            try:
#                new_terraforming_lists[idx].append(child_list[gen])
#            except IndexError:
#                continue
#        ave_terraforming_lists[idx].append(round(sum(new_terraforming_lists[idx]) / 
#                                                 len(new_terraforming_lists[idx])))
#        
#print(ave_ocean_gen_list)
#print(ave_oxygen_gen_list)
#print(ave_temperature_gen_list)
#
#print(temperature_gen_lists[2])
#print(temperature_gen_lists[4])

In [ ]:
## main value calculation function
#def base_card_value(database, generation, player_count):
#    income, vp, TR, last_gen = initialize(generation, player_count)
#    
#  # iterate through cards in database
#    for row in database.itertuples():
#        dict_row = row._asdict()
#        value = 0
#        
#    # Primary Cost------
#        value -= dict_row['Primary_Cost']
#        
#    # Additional Cost------
#        add_cost = dict_row['Additional_Cost']
#        if not isinstance(add_cost, float):
#            add_cost_lists = []
#            for idx, ele in enumerate(add_cost.split(',')):
#                add_cost_lists.append(ele.strip().split(' '))
#            for add_cost_list in add_cost_lists:
#                value -= vars_scan(add_cost_list, generation, player_count)
#                            
#    # Victory Points------
#        victory = dict_row['Victory_Points']
#        if not isinstance(victory, float):
#            if len(victory) <= 2:
#                value += int(victory) * vp 
#                
#    # Immediate Benefit------
#        i_benefit = dict_row['Immediate_Benefit']
#        if not isinstance(i_benefit, float):
#            i_benefit_lists = []
#            for idx, ele in enumerate(i_benefit.split(',')):
#                i_benefit_lists.append(ele.strip().split(' '))
#            for idx, i_benefit_list in enumerate(i_benefit_lists):
#                
#            # for benefits dependent on the amount of another variable
#                if 'per' in i_benefit_list:
#                    value += pers(i_benefit_list, generation, player_count)
#                
#            # for benefits with a choice; the larger of the two values is taken
#                elif 'or' in i_benefit_list:
#                    value += ors((' ').join(i_benefit_list), generation, player_count)
#                    
#            # for ocean and greenery tiles played on the opposite site
#               elif 'land' in i_benefit_list and 'ocean' in i_benefit_list:
#                    value += ocean + tile_land - tile_ocean
#                elif 'ocean' in i_benefit_list and 'greenery' in i_benefit_list:
#                    value += greenery + tile_ocean - tile_land
#                    
#            # for special tiles
#                elif 'special' in i_benefit_list:
#                    if 'ocean' in i_benefit_list:
#                        value += tile_ocean
#                    else:
#                        value += tile_land
#                
#            # for simple added values
#                else:
#                    value += vars_scan(i_benefit_list, generation, player_count)
#                    
#    # Active Cost/Benefit------
#        act_cost = dict_row['Active_Cost']
#        a_benefit = dict_row['Active_Benefit']
#        current_gen = generation
#        if not isinstance(act_cost, float) or not isinstance(a_benefit, float):
#            while current_gen <= last_gen:
#        
#            # Cost
#                cost, benefit = [0, 0]
#                if not isinstance(act_cost, float):
#                    act_cost_list = act_cost.split(' ')
#            
#                    if 'or' in act_cost_list:
#                        cost = ors(act_cost, current_gen, player_count, cost=True)
#                
#                    else:
#                        if any([x for x in a_benefit.split(' ') if x in vars_dict_variable]):
#                            if current_gen < last_gen:
#                                cost = vars_scan(act_cost_list, current_gen, player_count)
#                        else:
#                            cost = vars_scan(act_cost_list, current_gen, player_count)
#        
#            # Benefit
#                if not isinstance(a_benefit, float):
#                    benefit = 0
#                    for benefit_list in a_benefit.split(','):
#                        benefit_list_split = benefit_list.strip().split(' ')
#            
#                        if 'or' in benefit_list:
#                            benefit += ors(benefit_list, current_gen, player_count)
#                
#                        elif 'on card' in benefit_list:
#                            number = (int(victory[0]) / int(victory[6])) * int(benefit_list[0])
#                            current_vp = credits_per_vp(current_gen, player_count)
#                            benefit += number * current_vp
#                
#                        else:
#                            if any([x for x in benefit_list_split if x in \
#                                   [ele for ele in vars_dict_variable if ele \
#                                   not in ['CREDIT', 'PLANT']]]):
#                            # ensures increased production on the final turn isn't converted into points
#                                if current_gen < last_gen:
#                                    benefit += vars_scan(benefit_list_split, current_gen, player_count)
#                            else:
#                                benefit += vars_scan(benefit_list_split, current_gen, player_count)
#                                
#                if cost < benefit: value += benefit - cost
#                current_gen += 1
#                
#    # Removal from opponent------
#        removed = dict_row['Removed_from_Opponent']
#        if not isinstance(removed, float):
#            
#            if 'or' in removed:
#                value += ors(removed, generation, player_count)
#            
#            else:
#                value += vars_scan(removed.split(' '), generation, player_count)
#
#        database.loc[dict_row['Index'], 'Value'] = round(value)
#        database.loc[dict_row['Index'], 'Current_Generation'] = generation
#        database.loc[dict_row['Index'], 'Players'] = player_count
#    database.Value = database.Value.astype(int)
#    database.Current_Generation = database.Current_Generation.astype(int)
#    database.Players = database.Players.astype(int)
#    
#    return database
#            
#base_card_value(base_cases, 11, 2)

### DEBUGGER
# main value calculation function
def base_card_value(database, generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
  # iterate through cards in database
    for row in database.itertuples():
        dict_row = row._asdict()
        value = 0
        print('Card - ', dict_row['Title'])
        
    # Primary Cost------
        value -= dict_row['Primary_Cost']
        print('Primary Cost = ', dict_row['Primary_Cost'])
        
    # Additional Cost------
        add_cost = dict_row['Additional_Cost']
        if not isinstance(add_cost, float):
            add_cost_lists = []
            for idx, ele in enumerate(add_cost.split(',')):
                add_cost_lists.append(ele.strip().split(' '))
            for add_cost_list in add_cost_lists:
                value -= vars_scan(add_cost_list, generation, player_count)
            print('Additional Cost = ', vars_scan(add_cost_list, generation, player_count))
        else: print('Additional Cost = None')
                            
    # Victory Points------
        victory = dict_row['Victory_Points']
        if not isinstance(victory, float):
            if len(victory) <= 2:
                value += int(victory) * vp 
                print('Victory Points = ', int(victory) * vp)
            else: print('VP per calculated in Active Benefits')
        else: print('Victory Points = None')
                
    # Immediate Benefit------
        i_benefit = dict_row['Immediate_Benefit']
        if not isinstance(i_benefit, float):
            i_benefit_lists = []
            for idx, ele in enumerate(i_benefit.split(',')):
                i_benefit_lists.append(ele.strip().split(' '))
            for idx, i_benefit_list in enumerate(i_benefit_lists):
                
            # for benefits dependent on the amount of another variable
                if 'per' in i_benefit_list:
                    value += pers(i_benefit_list, generation, player_count)
                    print('Immediate Per Points = ', pers(i_benefit_list, generation, player_count))
                
            # for benefits with a choice; the larger of the two values is taken
                elif 'or' in i_benefit_list:
                    value += ors((' ').join(i_benefit_list), generation, player_count)
                    print('Immediate Or Points = ', ors((' ').join(i_benefit_list), generation, player_count))
                    
            # for ocean and greenery tiles played on the opposite site
                elif 'land' in i_benefit_list and 'ocean' in i_benefit_list:
                    value += ocean + tile_land - tile_ocean
                    print('Opposite Land Type = ', ocean + tile_land - tile_ocean)
                elif 'ocean' in i_benefit_list and 'greenery' in i_benefit_list:
                    value += greenery + tile_ocean - tile_land
                    print('Opposite Land Type = ', greenery + tile_ocean - tile_land)
                    
            # for special tiles
                elif 'special' in i_benefit_list:
                    if 'ocean' in i_benefit_list:
                        value += tile_ocean
                        print('Special Tile Ocean = ', tile_ocean)
                    else:
                        value += tile_land
                        print('Special Tile Land = ', tile_land)
                
            # for simple added values
                else:
                    value += vars_scan(i_benefit_list, generation, player_count)
                    print('Immediate Benefit = ', vars_scan(i_benefit_list, generation, player_count))
        else: print ('Immediate Benefit = None')
                    
    # Active Cost/Benefit------
        act_cost = dict_row['Active_Cost']
        a_benefit = dict_row['Active_Benefit']
        current_gen = generation
        if isinstance(act_cost, float) and isinstance(a_benefit, float):
            print('Active Cost and Benefit = None')
        else:
            total_val = 0
            while current_gen <= last_gen:
                print('***Current Gen = ', current_gen, ' ***')
        
            # Cost
                cost, benefit = [0, 0]
                if not isinstance(act_cost, float):
                    act_cost_list = act_cost.split(' ')
            
                    if 'or' in act_cost_list:
                        cost = ors(act_cost, current_gen, player_count, cost=True)
                        print('Active Or Cost = ', ors(act_cost, current_gen, player_count, cost=True))
                
                    else:
                        if any([x for x in a_benefit.split(' ') if x in vars_dict_variable]):
                            if current_gen < last_gen:
                                cost = vars_scan(act_cost_list, current_gen, player_count)
                                print('Active Cost = ', vars_scan(act_cost_list, current_gen, player_count))
                        else:
                            cost = vars_scan(act_cost_list, current_gen, player_count)
                            print('Active Cost = ', vars_scan(act_cost_list, current_gen, player_count))
                else: print('Active Cost = None')
        
            # Benefit
                if not isinstance(a_benefit, float):
                    benefit = 0
                    for benefit_list in a_benefit.split(','):
                        benefit_list_split = benefit_list.strip().split(' ')
            
                        if 'or' in benefit_list:
                            benefit += ors(benefit_list, current_gen, player_count)
                            print('Active Or Benefit = ', ors(benefit_list, current_gen, player_count))
                
                        elif 'on card' in benefit_list:
                            number = (int(victory[0]) / int(victory[6])) * int(benefit_list[0])
                            current_vp = credits_per_vp(current_gen, player_count)
                            print('current_vp = ', current_vp)
                            benefit += number * current_vp
                            print('Active On Card Benefit = ', number * current_vp)
                
                        else:
                            if any([x for x in benefit_list_split if x in \
                                   [ele for ele in vars_dict_variable if ele \
                                   not in ['CREDIT', 'PLANT']]]):
                            # ensures increased production on the final turn isn't converted into points
                                if current_gen < last_gen:
                                    benefit += vars_scan(benefit_list_split, current_gen, player_count)
                                    print('Active Benefit Variable = ', vars_scan(benefit_list_split, current_gen, \
                                                                        player_count))
                            else:
                                benefit += vars_scan(benefit_list_split, current_gen, player_count)
                                print('Active Benefit Static = ', vars_scan(benefit_list_split, current_gen, player_count))
                else: print('Active Benefit = None')
                if cost < benefit: value += benefit - cost
                print('Net Active Points = ', benefit - cost)
                total_val += benefit - cost
                current_gen += 1
                print('Total Active Cost/Benefit = ', total_val)
                
    # Removal from opponent------
        removed = dict_row['Removed_from_Opponent']
        if not isinstance(removed, float):
            
            if 'or' in removed:
                value += ors(removed, generation, player_count)
                print('Or Removal from Opponent = ', ors(removed, generation, player_count))
            
            else:
                value += vars_scan(removed.split(' '), generation, player_count)
                print('Removal = ', vars_scan(removed.split(' '), generation, player_count))
        else: print('Removal from Opponenet = None')

        database.loc[dict_row['Index'], 'Value'] = round(value)
        database.loc[dict_row['Index'], 'Current_Generation'] = generation
        database.loc[dict_row['Index'], 'Players'] = player_count
    database.Value = database.Value.astype(int)
    database.Current_Generation = database.Current_Generation.astype(int)
    database.Players = database.Players.astype(int)
            
base_card_value(base_cases, 11, 2)
display(base_cases)